#### **Developer**: Pedro Augusto Andrade Silva
#### **Objectives**: Outlook automation of e-mail sending with different attachments and recipient list, considering the business days and with a customizable e-mail body.

## 1) Libraries used

In [17]:
import calendar
import locale
import time
from datetime import datetime, timedelta, date
import win32com.client as win32
import pandas as pd
import holidays

## 2) Initial settings

In [18]:
locale.setlocale(locale.LC_ALL, "pt_BR.utf8")
current_month_name = calendar.month_name[datetime.today().month].title()


br_holidays = holidays.Brazil(years = datetime.today().year)
holidays = [date for date, _ in sorted(br_holidays.items())]

file_name = [
    'attachment1',
    'attachment2',
    'attachment3',
]

attachments = [
    r"C:\Users\pedro\OneDrive\Área de Trabalho\Pedro\Projetos\automacoes\outlook\attachment1.xlsx",
    r"C:\Users\pedro\OneDrive\Área de Trabalho\Pedro\Projetos\automacoes\outlook\attachment2.xlsx",
    r"C:\Users\pedro\OneDrive\Área de Trabalho\Pedro\Projetos\automacoes\outlook\attachment3.xlsx",
]


## 3) Functions

In [19]:
def calculate_delivery_deadline(current_date, holidays):
    working_days = 0
    while working_days < 2:
        delivery_date = current_date + timedelta(days=1)
        if delivery_date.weekday() < 5 and delivery_date not in holidays:
            working_days += 1
        current_date = delivery_date
    return delivery_date

def success_message(file_name):
    time.sleep(1)
    print(f"The file \"{file_name}\" has been successfully sent!")

current_date = datetime.now().date()
delivery_date = calculate_delivery_deadline(current_date, holidays)
delivery_date_formated = delivery_date.strftime("%d-%m")

## 4) Execution

In [20]:
outlook = win32.Dispatch('outlook.application')

with open("email_body.txt", "r", encoding="utf-8") as txt_file:
    personalized_text = txt_file.read()

for i in range(3):
    df_emails_to = pd.read_excel(r"C:\Users\pedro\OneDrive\Área de Trabalho\Pedro\Projetos\automacoes\outlook\recipients.xlsx", sheet_name=0, header=0)
    df_emails_cc = pd.read_excel(r"C:\Users\pedro\OneDrive\Área de Trabalho\Pedro\Projetos\automacoes\outlook\recipients.xlsx", sheet_name=1, header=0)
    df_emails_bcc = pd.read_excel(r"C:\Users\pedro\OneDrive\Área de Trabalho\Pedro\Projetos\automacoes\outlook\recipients.xlsx", sheet_name=2, header=0)
    recipient_to = list(df_emails_to[f"Grupo {i+1}"])
    recipient_cc = list(df_emails_cc[f"Grupo {i+1}"])
    recipien_bcc = list(df_emails_bcc[f"Grupo {i+1}"])

    email = outlook.CreateItem(0)
    email.To = ';'.join(recipient_to)
    email.CC = ';'.join(recipient_cc)
    email.BCC = ';'.join(recipien_bcc)
    email.Subject = f"E-mails sending - {current_month_name}/23"
    email.Body = f"""Dear,

Please send us the document within a period of up to 2 business days, that is, by the day {delivery_date_formated}.

{personalized_text}"""

    email.Attachments.Add(attachments[i])
    email.Send()
    
    success_message(file_name[i])

The file "attachment1" has been successfully sent!
The file "attachment2" has been successfully sent!
The file "attachment3" has been successfully sent!
